## **Final Term project .**



In [ ]:
!pip install rank_bm25

In [ ]:
### Google Drive 패키지와 os 모듈 불러오기
from google.colab import drive
import os

In [ ]:
### 본인 구글 드라이브의 최초 경로를 설정하기

# 대부분의 구글 드라이브 최초 경로는 아래와 같습니다. 
# 예외 발생 시 본인의 구글 드라이브에 접속하여 content 폴더나 gdrive 폴더가 어떤 위치에 있으며, 
# 자신이 연결하려는 폴더까지의 경로가 어떻게 되는지 확인해야 합니다.  
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
### 구글 클라우드 컴퓨터에 elastic server 서버 설치를 위한 폴더 생성 
!sudo mkdir /content/elasticsearch
### 접근 권한 수정
!chmod 755 -R elasticsearch
### 현재 작업 디렉토리 설정
os.chdir('/content/elasticsearch')

In [ ]:
### 리눅스용 엘라스틱서치 서버 설치를 위한 패키지 다운로드
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.0.0-linux-x86_64.tar.gz -q

In [ ]:
### 위에서 다운로드 받은 압축 파일을 해제
!tar -xzf elasticsearch-7.0.0-linux-x86_64.tar.gz

In [ ]:
### 코랩 노트북 환경에서 서버 구동을 위해서 PPID 1의 백그라운드 데몬 프로세스가 해당 폴더에 접근이 가능하도록 소유자 변경
!chown -R daemon:daemon elasticsearch-7.0.0

In [ ]:
### 파이썬 환경에서 구동을 위한 elasticsearch 패키지 설치
!pip install elasticsearch

     |████████████████████████████████| 358kB 13.7MB/s eta 0:00:01


In [ ]:
# 데몬 프로세스로 엘라스틱 서버 개시하기
import os
from subprocess import Popen, PIPE, STDOUT
es = Popen(['elasticsearch-7.0.0/bin/elasticsearch'], 
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                )

In [ ]:
# 로컬 서버에 엘라스틱 서버와 python을 연결
from elasticsearch import Elasticsearch
es = Elasticsearch("localhost:9200/")
es.info()

{'cluster_name': 'elasticsearch',
 'cluster_uuid': 'lBCydBeXQOimIjYNClO-mQ',
 'name': 'ff6d2b4dba25',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2019-04-05T22:55:32.697037Z',
  'build_flavor': 'default',
  'build_hash': 'b7e28a7',
  'build_snapshot': False,
  'build_type': 'tar',
  'lucene_version': '8.0.0',
  'minimum_index_compatibility_version': '6.0.0-beta1',
  'minimum_wire_compatibility_version': '6.7.0',
  'number': '7.0.0'}}

In [ ]:
### 현재 작업 환경 재설정하기 (자신만의 작업 디렉토리를 기본 경로로 설정하는 것)

#.ipynb 노트북 파일과 불러오려는 이미지 및 모듈에 해당하는 파일들은 같은 현재 작업 환경 
#또는 그 하위 폴더에 존재하는 것이 좋습니다. 

# 자신만의 기본 경로를 설정하는데, 대부분의 경우에는 Colab Notebooks까지 동일하게 공유하며,
# 이후 경로는 자신이 생성한 폴더에 맞추어서 경로를 설정하면 됩니다.
# 수업에서는 자신의 구글 드라이브에 information_retrieval 폴더를 생성하시면 됩니다. 
os.chdir('/content/gdrive/My Drive/Colab Notebooks/information_retrieval/elastic')

In [ ]:
# 데이터를 색인화하기 위한 함수
def indexing(es, index_name):
    # 이미 존재할 경우 삭제하고 다시 만들기
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)

    # 인덱스 생성
    print(es.indices.create(index=index_name))

# 인덱스명을 정하기 (자유롭게)
index_name = 'sample_index'
indexing(es, index_name)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'sample_index'}


In [ ]:
### 새로운 데이터 인덱싱 
## 여기서는 '노래 제목'과 '가사 일부'를 mapping했습니다.
sample1= {'songs': '민족의 아리아', 'lyrics': '지축을 박차고 포효하라 그대'}
sample2= {'songs': '뱃노래', 'lyrics': '즐거운 고연전 날에 지고 가는 연대생이 처량도 하구나'}
sample3= {'songs': '석탑', 'lyrics': '이름모를 석공의 땀과 눈물이 흘러내리네'}
sample4= {'songs': 'Forever', 'lyrics': '우리의 함성은 신화가 되리라 울려라 이곳에 Forever'}
sample5= {'songs': '젊은그대', 'lyrics': '사랑스런 젊은 그대 태양같은 젊은 그대'}

# es 변수에 객체화한 서버는 위에서 정의한 데아터에 대해서
# 이전 셀에서 선언한 'sample_index'로 인덱스명을 정하고
# 데이터의 타입은 문자열인 상태로 색인화를 진행합니다.
es.index(index=index_name, doc_type='string', body = sample1)
es.index(index=index_name, doc_type='string', body = sample2)
es.index(index=index_name, doc_type='string', body = sample3)
es.index(index=index_name, doc_type='string', body = sample4)
es.index(index=index_name, doc_type='string', body = sample5)

es.indices.refresh(index=index_name)

/usr/local/lib/python3.7/dist-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: [types removal] Specifying types in document index requests is deprecated, use the typeless endpoints instead (/{index}/_doc/{id}, /{index}/_doc, or /{index}/_create/{id}).
  warnings.warn(message, category=ElasticsearchWarning)


{'_shards': {'failed': 0, 'successful': 1, 'total': 2}}

In [ ]:
# 검색 해보기
results = es.search(index=index_name, body={'from':0, 'size':10, 'query':{'match':{'lyrics':'그대'}}})

# 딕셔너리 안에 딕셔너리: 'hits'라는 key 안에 value는 또 다른 사전형으로 되어있으며, 해당 사전의 key 이름은 'hits'   
for result in results['hits']['hits']:
    print('score:', result['_score'], 'source::', result['_source'])

score: 1.2037696 source:: {'songs': '젊은그대', 'lyrics': '사랑스런 젊은 그대 태양같은 젊은 그대'}
score: 1.0137007 source:: {'songs': '민족의 아리아', 'lyrics': '지축을 박차고 포효하라 그대'}


In [ ]:
# 샘플 영어 위키 덤프를 다운로드
!wget https://github.com/AlonEirew/wikipedia-to-elastic/raw/master/dumps/tinywiki-latest-pages-articles.xml.bz2

--2021-06-25 11:05:45--  https://github.com/AlonEirew/wikipedia-to-elastic/raw/master/dumps/tinywiki-latest-pages-articles.xml.bz2
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/AlonEirew/wikipedia-to-elastic/master/dumps/tinywiki-latest-pages-articles.xml.bz2 [following]
--2021-06-25 11:05:45--  https://raw.githubusercontent.com/AlonEirew/wikipedia-to-elastic/master/dumps/tinywiki-latest-pages-articles.xml.bz2
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9731 (9.5K) [application/octet-stream]
Saving to: ‘tinywiki-latest-pages-articles.xml.bz2.1’

tinywiki-latest-pag 100%[===================

In [ ]:
# 위키데이터의 노이즈를 제거하고 json 형태로 반환하는 코드를 참조
!git clone https://github.com/attardi/wikiextractor.git

fatal: destination path 'wikiextractor' already exists and is not an empty directory.


In [ ]:
# 다운로드 받은 샘플 위키 데이터를 전처리하여 검색의 입력으로 사용
# 결과는 elastic 폴더에 'extract_result/AA'라는 새로운 폴더에 저장된다.
!python -m wikiextractor.wikiextractor.WikiExtractor tinywiki-latest-pages-articles.xml.bz2 --json -o extract_result

INFO: Preprocessing 'tinywiki-latest-pages-articles.xml.bz2' to collect template definitions: this may take some time.
INFO: Loaded 0 templates in 0.0s
INFO: Starting page extraction from tinywiki-latest-pages-articles.xml.bz2.
INFO: Using 1 extract processes.
INFO: Finished 1-process extraction of 7 articles in 0.0s (182.4 art/s)


In [ ]:
# 엘라스틱서치 서버와 마찬가지로 구글 드라이브 내부 보안 정책으로 인해,
# DRQA의 필수 설치 패키지인 CoreNLP 서버 연동을 위해서 구글 드라이브 바깥의 구글 클라우드 컴퓨터에 저장소 코드를 다운 받습니다.
os.chdir('/content/elasticsearch/')
!git clone https://github.com/facebookresearch/DrQA.git

Cloning into 'DrQA'...
remote: Enumerating objects: 269, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 269 (delta 0), reused 1 (delta 0), pack-reused 265
Receiving objects: 100% (269/269), 564.11 KiB | 2.29 MiB/s, done.
Resolving deltas: 100% (127/127), done.


In [ ]:
# 내려받은 저장소 패키지 안에 필수 항목 설치 
os.chdir('/content/elasticsearch/DrQA')
!pip install -r requirements.txt
!python setup.py develop

     |████████████████████████████████| 61kB 7.3MB/s 
  Found existing installation: pexpect 4.8.0
    Uninstalling pexpect-4.8.0:
      Successfully uninstalled pexpect-4.8.0


running develop
running egg_info
creating drqa.egg-info
writing drqa.egg-info/PKG-INFO
writing dependency_links to drqa.egg-info/dependency_links.txt
writing requirements to drqa.egg-info/requires.txt
writing top-level names to drqa.egg-info/top_level.txt
writing manifest file 'drqa.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'drqa.egg-info/SOURCES.txt'
running build_ext
Creating /usr/local/lib/python3.7/dist-packages/drqa.egg-link (link to .)
Adding drqa 0.1.0 to easy-install.pth file

Installed /content/elasticsearch/DrQA
Processing dependencies for drqa==0.1.0
Searching for pexpect==4.2.1
Best match: pexpect 4.2.1
Adding pexpect 4.2.1 to easy-install.pth file

Using /usr/local/lib/python3.7/dist-packages
Searching for elasticsearch==7.13.1
Best match: elasticsearch 7.13.1
Adding elasticsearch 7.13.1 to easy-install.pth file

Using /usr/local/lib/python3.7/dist-packages
Searching for nltk==3.2.5
Best match: nltk 3.2.5
Adding nltk 3.2.5 to easy-install.pt

In [ ]:
# CoreNLP 설치
!./install_corenlp.sh

Specify download path or enter to use default (data/corenlp): yes
Will download to: yes
/tmp /content/elasticsearch/DrQA
--2021-06-25 11:09:00--  http://nlp.stanford.edu/software/stanford-corenlp-full-2017-06-09.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/software/stanford-corenlp-full-2017-06-09.zip [following]
--2021-06-25 11:09:00--  https://nlp.stanford.edu/software/stanford-corenlp-full-2017-06-09.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://downloads.cs.stanford.edu/nlp/software/stanford-corenlp-full-2017-06-09.zip [following]
--2021-06-25 11:09:01--  https://downloads.cs.stanford.edu/nlp/software/stanford-corenlp-full-2017-06-09.zip
Resolving downloads.cs.stanford.edu (downloads.cs.st

In [ ]:
# python 3.6과 3.7에서의 충돌을 해결하기 위한 코드
# 가볍게 실행하고 넘어가기
!sudo apt-get remove python-pexpect python3-pexpect
!sudo pip3.7 install --upgrade pexpect

In [ ]:

os.chdir('/content/elasticsearch/DrQA/scripts/retriever/')
!python build_db.py '/content/gdrive/My Drive/Colab Notebooks/information_retrieval/elastic/extract_result/AA/wiki_00' '/content/gdrive/My Drive/Colab Notebooks/information_retrieval/elastic/sample_wiki.db'

Traceback (most recent call last):
  File "build_db.py", line 18, in <module>
    from drqa.retriever import utils
  File "/content/elasticsearch/DrQA/drqa/__init__.py", line 20, in <module>
    from . import tokenizers
  File "/content/elasticsearch/DrQA/drqa/tokenizers/__init__.py", line 20, in <module>
    from .corenlp_tokenizer import CoreNLPTokenizer
  File "/content/elasticsearch/DrQA/drqa/tokenizers/corenlp_tokenizer.py", line 14, in <module>
    import pexpect
  File "/usr/local/lib/python3.7/dist-packages/pexpect/__init__.py", line 75, in <module>
    from .pty_spawn import spawn, spawnu
  File "/usr/local/lib/python3.7/dist-packages/pexpect/pty_spawn.py", line 14, in <module>
    from .spawnbase import SpawnBase
  File "/usr/local/lib/python3.7/dist-packages/pexpect/spawnbase.py", line 224
    def expect(self, pattern, timeout=-1, searchwindowsize=-1, async=False):
                                                                   ^
SyntaxError: invalid syntax


In [ ]:
os.chdir('/content/elasticsearch/DrQA')

In [ ]:
import drqa.tokenizers
import drqa.reader

from drqa.retriever.doc_db import DocDB
from elasticsearch import Elasticsearch
from tqdm import tqdm
from drqa.tokenizers import CoreNLPTokenizer
from drqa.pipeline import DrQA
from drqa.retriever import ElasticDocRanker, TfidfDocRanker

# 새로운 인덱스 생성
def make_index(es, index_name):
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
    print(es.indices.create(index=index_name))

# 인덱스 명은 자유롭게
index_name = 'test_index'
make_index(es, index_name)

# python build_db.py로 생성한 db의 경로
db = DocDB(db_path='/content/gdrive/My Drive/Colab Notebooks/information_retrieval/elastic/sample_wiki.db')
print(len(db.get_doc_ids())) # 6개

# 색인화 진행
for idx, doc_id in enumerate(tqdm(db.get_doc_ids())):
    doc = {'title': doc_id, 'content': db.get_doc_text(doc_id)}
    es.index(index = index_name, doc_type= 'string', body=doc)

es.indices.refresh(index=index_name)

# 색인된 엘라스틱서치 데이터베이스에서 'content'를 key로 하여, value에 질의를 입력하여 검색 (여기서는 film을 질의로 사용)
results = es.search(index=index_name, body={'from':0, 'size':10, 'query':{'match':{'content':'film'}}})
for result in results['hits']['hits']:
    print('score:', result['_score'], 'source:', result['_source'])

06/25/2021 12:36:43 - INFO - elasticsearch -   HEAD http://localhost:9200/test_index [status:200 request:0.010s]
06/25/2021 12:36:43 - INFO - elasticsearch -   DELETE http://localhost:9200/test_index [status:200 request:0.040s]
06/25/2021 12:36:44 - INFO - elasticsearch -   PUT http://localhost:9200/test_index [status:200 request:0.128s]
  0%|          | 0/6 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: [types removal] Specifying types in document index requests is deprecated, use the typeless endpoints instead (/{index}/_doc/{id}, /{index}/_doc, or /{index}/_create/{id}).
  warnings.warn(message, category=ElasticsearchWarning)
06/25/2021 12:36:44 - INFO - elasticsearch -   POST http://localhost:9200/test_index/string [status:201 request:0.039s]
06/25/2021 12:36:44 - INFO - elasticsearch -   POST http://localhost:9200/test_index/string [status:201 request:0.010s]
06/25/2021 12:36:44 - INFO - elasticsearch -   POST htt

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'test_index'}
6
score: 1.8695081 source: {'title': '4', 'content': "The Royal Cinema is an Art Moderne event venue and cinema in Toronto, Canada. It was built in 1939 and owned by Miss Ray Levinsky.\nWhen it was built in 1939, it was called The Pylon, with an accompanying large sign at the front of the theatre. It included a roller-skating rink at the rear of the theatre, and a dance hall on the second floor.\nIn the 1950's the theatre was purchased by Rocco Mastrangelo. In the 1990s, the theatre was renamed 'the Golden Princess'.\nSince early 2007, Theatre D has owned and operated The Royal.\nDuring the daytime it operates as a film and television post-production studio. It hosts film festivals including the European Union Film Festival and Japanese Movie Week.\nThe Royal was featured in the 2013 film The F Word.\nReferences.\n "}


In [ ]:

os.chdir('/content/gdrive/My Drive/Colab Notebooks/information_retrieval(2)') ## 현재 작업 환경으로 설정한 경로를 입력하세요

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Jun 25 11:14:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### **필요한 패키지 불러오기** 😼


In [ ]:
# 설치하는 항목들
!pip install datasets
!pip install transformers
!pip install accelerate

# 로드하는 항목들
import argparse
import math
import os
import random
import datasets
import numpy as np
import torch
import collections
import json
import logging
import transformers
import easydict
import pandas as pd

from tqdm import tqdm_notebook
from tqdm.auto import tqdm
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets, load_metric
from torch.utils.data.dataloader import DataLoader
from typing import Optional, Tuple
from accelerate import Accelerator

# Huggingface transformers의 토크나이저, 설정파일, 최적화 함수 등 사용
from transformers import (
    CONFIG_MAPPING,
    MODEL_MAPPING,
    AdamW,
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    SchedulerType,
    default_data_collator,
    get_scheduler,
    set_seed,
)
from utils_qa import postprocess_qa_predictions
from transformers.utils import check_min_version

     |████████████████████████████████| 245kB 14.3MB/s eta 0:00:01
     |████████████████████████████████| 245kB 22.1MB/s 
     |████████████████████████████████| 122kB 30.6MB/s 
     |████████████████████████████████| 2.5MB 14.6MB/s 
     |████████████████████████████████| 3.3MB 52.8MB/s 
     |████████████████████████████████| 901kB 44.5MB/s 
     |████████████████████████████████| 51kB 2.9MB/s 


In [ ]:
## 명령행 인터페이스, 사전 훈련된 모델을 로드하여 훈련하고 평가하는데까지 필요한 인자를 사용자 정의 인자로 설정하는 내용을 담고 있음.
## 사용법: python example.py --dataset_name squad.json .... 
## 주피터 환경을 위한 인자 설정법 
import easydict
import string
import re
def easydict_args():
    args = easydict.EasyDict({
        "dataset_name": 'squad',         
        "dataset_config_name": 'plain_text', 
#        "dataset_name": None,         
#        "dataset_config_name": 'squad', 
        "train_file": 'final_test_no_context.json',
        "preprocessing_num_workers": 4,
        "validation_file": 'final_test_no_context.json',
        "test_file": 'final_test_no_context.json',
        "test_save_path" : "result_final/final_test.csv",
        "max_seq_length": 384,
        "pad_to_max_length": None,
        "model_name_or_path": 'result/pytorch_model.bin',
        "config_name": 'bert-base-cased',
        "tokenizer_name": 'bert-base-cased',
        "use_slow_tokenizer": None,
        "per_device_train_batch_size": 4,
        "per_device_eval_batch_size": 4,
        "learning_rate": 3e-5,
        "weight_decay": 0.0,
        "num_train_epochs": 3,
        "max_train_steps": None,
        "gradient_accumulation_steps": 1,
        "lr_scheduler_type": 'linear',
        "num_warmup_steps":0,
        "output_dir": 'result_final/',
        "seed": 42,
        "doc_stride": 128,
        "n_best_size": 20,
        "null_score_diff_threshold": 0.0,
        "version_2_with_negative": False,
        "max_answer_length": 30,
        "max_train_samples": None,
        "max_eval_samples": None,
        "overwrite_cache": False,
        "max_predict_samples": 30,
        "do_train": False,
        "do_eval": False,        
        "do_predict": False,
        "do_test" : True
    })

    ## 무결성 체크
    ## 인자로 전달한 데이터셋 명이 잘못되었거나 확장자 오류가 있을 경우 raise 이하의 메세지를 출력
    if (
        args.dataset_name is None
        and args.train_file is None
        and args.validation_file is None
        and args.test_file is None
    ):
        raise ValueError("Need either a dataset name or a training/validation/test file.")
    else:
        if args.train_file is not None:
            extension = args.train_file.split(".")[-1]
            assert extension in ["csv", "json"], "`train_file` should be a csv or a json file."
        if args.validation_file is not None:
            extension = args.validation_file.split(".")[-1]
            assert extension in ["csv", "json"], "`validation_file` should be a csv or a json file."
        if args.test_file is not None:
            extension = args.test_file.split(".")[-1]
            assert extension in ["csv", "json"], "`test_file` should be a csv or a json file."

    if args.output_dir is not None:
        os.makedirs(args.output_dir, exist_ok=True)
    
    ## 위에서 인자로 전달받은 값을 args 변수에 담아 반환
    return args

In [ ]:
# SQuAD 평가를 위한 전처리 함수
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""

    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [ ]:
logger = logging.getLogger(__name__)
def main():
    args = easydict_args()

    # Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
    ## 가속화 모드를 지원하며, 해당 모드로 병목이 발생하는 메서드 부분을 대체해서 사용가능.
    accelerator = Accelerator()
    # Make one log on every process with the configuration for debugging.
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
    )
    logger.info(accelerator.state)

    # Setup logging, we only want one process per machine to log things on the screen.
    # accelerator.is_local_main_process is only True for one process per machine.
    logger.setLevel(logging.INFO if accelerator.is_local_main_process else logging.ERROR)
    if accelerator.is_local_main_process:
        datasets.utils.logging.set_verbosity_warning()
        transformers.utils.logging.set_verbosity_info()
    else:
        datasets.utils.logging.set_verbosity_error()
        transformers.utils.logging.set_verbosity_error()

    # If passed along, set the training seed now.
    if args.seed is not None:
        set_seed(args.seed)

    # Get the datasets: you can either provide your own CSV/JSON/TXT training and evaluation files (see below)
    # or just provide the name of one of the public datasets available on the hub at https://huggingface.co/datasets/
    # (the dataset will be downloaded automatically from the datasets Hub).
    #
    # For CSV/JSON files, this script will use the column called 'text' or the first column if no column called
    # 'text' is found. You can easily tweak this behavior (see below).
    #
    # In distributed training, the load_dataset function guarantee that only one local process can concurrently
    # download the dataset.
    
    ## Huggingface의 datasets에 존재하는 데이터셋 명을 사용하는 경우에는 저장된 형식에 맞추어서 로드 (위의 링크에서 리스트 참조)
    if args.dataset_name is not None:
        # Downloading and loading a dataset from the hub.
        print("========== load dataset from HUB ============")
        raw_datasets = load_dataset(args.dataset_name, args.dataset_config_name)
        print(raw_datasets,args.dataset_name, args.dataset_config_name )
    ## 사용자 정의의 데이터셋을 사용하는 경우에는 아래의 조건문 실행. 
    else:
        print("========== load dataset from User dataset ============")
        data_files = {}
        if args.train_file is not None:
            data_files["train"] = args.train_file
        if args.validation_file is not None:
            data_files["validation"] = args.validation_file
        if args.test_file is not None:
            data_files["test"] = args.test_file
        #extension = args.train_file.split(".")[-1]
        extension = 'json'
        print("data_files",data_files)
        raw_datasets = load_dataset(extension, data_files=data_files, field="data")
        print("raw_datasets",raw_datasets)

    
    # See more about loading any type of standard or custom dataset (from files, python dict, pandas DataFrame, etc) at
    # https://huggingface.co/docs/datasets/loading_datasets.html.

    # Load pretrained model and tokenizer
    #
    # In distributed training, the .from_pretrained methods guarantee that only one local process can concurrently
    # download model & vocab.

    ## 지정한 모델의 설정에 맞추어서 환경 구성 요소 로드
    ## ex) BERT --> BERTConfig...
    print("========== Load Config  ============")
    if args.config_name:
        print("Load Config(1)")
        config = AutoConfig.from_pretrained(args.config_name)
    elif args.model_name_or_path:
        print("Load Config(2)")
        config = AutoConfig.from_pretrained(args.model_name_or_path)
    else:
        print("Load Config(3)")
        config = CONFIG_MAPPING[args.model_type]()
        logger.warning("You are instantiating a new config instance from scratch.")
    

    ## 지정한 모델의 설정에 맞추어서 토크나이저 로드
    ## ex) BERT --> BERTTokenizer ... 30,000개의 vocab와 word piece tokenzier를 곁들인...
    print("========== Load Tokenizer  ============")
    if args.tokenizer_name:
        print("Load Tokenizer(1)")
        tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, use_fast=True)
    elif args.model_name_or_path:
        print("Load Tokenizer(2)")
        tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, use_fast=True)
    else:
        raise ValueError(
            "You are instantiating a new tokenizer from scratch. This is not supported by this script."
            "You can do it from another script, save it, and load it from here, using --tokenizer_name."
        )

    ## 지정한 모델 로드하기.
    ## 여기서는 사전 훈련된 모델 상단에 질의응답 TASK를 위한 레이어를 추가하고 미세조정훈련을 하기위해 ForQuestionAnswering을 불러온다.
    if args.model_name_or_path:
        model = AutoModelForQuestionAnswering.from_pretrained(
            args.model_name_or_path,
            from_tf=bool(".ckpt" in args.model_name_or_path),
            config=config,
        )
    else:
        logger.info("Training new model from scratch")
        model = AutoModelForQuestionAnswering.from_config(config)

    # Preprocessing the datasets.
    # Preprocessing is slighlty different for training and evaluation.
    
 
    ## 전처리하기 
    print("========== Pre processing  0 ============")
#    column_names = raw_datasets["test"].column_names
#    column_names = raw_datasets["train"].column_names
    column_names = raw_datasets["train"].column_names
    print("========== Pre processing  1 ============")
    print("column_names",column_names)
    #print(column_names)
    question_column_name = "question" if "question" in column_names else column_names[0]
    context_column_name = "context" if "context" in column_names else column_names[1]
    answer_column_name = "answers" if "answers" in column_names else column_names[2]

    # Padding side determines if we do (question|context) or (context|question).
    pad_on_right = tokenizer.padding_side == "right"

    ## 인자로 전달한 최대 입력 시퀀스 길이가 모델의 최대 입력 임베딩 길이보다 길 경우 에러 반환
    if args.max_seq_length > tokenizer.model_max_length:
        logger.warning(
            f"The max_seq_length passed ({args.max_seq_length}) is larger than the maximum length for the"
            f"model ({tokenizer.model_max_length}). Using max_seq_length={tokenizer.model_max_length}."
        )

    max_seq_length = min(args.max_seq_length, tokenizer.model_max_length)
    

    ######################
    # Validation preprocessing
    def prepare_validation_features(examples):
        # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
        # in one example possible giving several features when a context is long, each of those features having a
        # context that overlaps a bit the context of the previous feature.
        tokenized_examples = tokenizer(
            examples[question_column_name if pad_on_right else context_column_name],
            examples[context_column_name if pad_on_right else question_column_name],
            truncation="only_second" if pad_on_right else "only_first",
            max_length=max_seq_length,
            stride=args.doc_stride,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length" if args.pad_to_max_length else False,
        )

        # Since one example might give us several features if it has a long context, we need a map from a feature to
        # its corresponding example. This key gives us just that.
        sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

        # For evaluation, we will need to convert our predictions to substrings of the context, so we keep the
        # corresponding example_id and we will store the offset mappings.
        tokenized_examples["example_id"] = []

        for i in range(len(tokenized_examples["input_ids"])):
            # Grab the sequence corresponding to that example (to know what is the context and what is the question).
            sequence_ids = tokenized_examples.sequence_ids(i)
            context_index = 1 if pad_on_right else 0

            # One example can give several spans, this is the index of the example containing this span of text.
            sample_index = sample_mapping[i]
            tokenized_examples["example_id"].append(examples["id"][sample_index])

            # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
            # position is part of the context or not.
            tokenized_examples["offset_mapping"][i] = [
                (o if sequence_ids[k] == context_index else None)
                for k, o in enumerate(tokenized_examples["offset_mapping"][i])
            ]

        return tokenized_examples

    #################
    ## 사용자 정의 검증 데이터를 입력 인자로 사용하는 경우, --do_eval을 입력 인자로 반드시 사용해야함.
    if args.do_eval:
        if "validation" not in raw_datasets:
            raise ValueError("--do_eval requires a validation dataset")
        eval_examples = raw_datasets["validation"]
        print("do_eval",eval_examples)
        if args.max_eval_samples is not None:
            # We will select sample from whole data
            eval_examples = eval_examples.select(range(args.max_eval_samples))
        # Validation Feature Creation
        eval_dataset = eval_examples.map(
            prepare_validation_features,
            batched=True,
            num_proc=args.preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=not args.overwrite_cache,
        )

        if args.max_eval_samples is not None:
            # During Feature creation dataset samples might increase, we will select required samples again
            eval_dataset = eval_dataset.select(range(args.max_eval_samples))

    #################
    ## 실제 정답을 반환하고 싶을 경우
    ## --do_predict을 입력 인자로 전달하는 경우, 사용자 정의 평가 데이터를 입력 인자로 전달해야함. 
    if args.do_predict:
 ##       if "test" not in raw_datasets:
 ##           print("do_predict - test is not in raw_datasets")
 ##           raise ValueError("--do_predict requires a test dataset")
 ##       predict_examples = raw_datasets["test"]
        predict_examples = raw_datasets["train"]
 ##       predict_examples = raw_datasets["validation"] 
        print("predict_examples(1)",predict_examples)
        if args.max_predict_samples is not None:
            # We will select sample from whole data
            predict_examples = predict_examples.select(range(args.max_predict_samples))
            print("predict_examples(2)",predict_examples)
 #           print("predict_examples.select",predict_examples.select.question)
        # Predict Feature Creation
        predict_dataset = predict_examples.map(
            prepare_validation_features,
            batched=True,
            num_proc=args.preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=not args.overwrite_cache,
        )
        if args.max_predict_samples is not None:
            # During Feature creation dataset samples might increase, we will select required samples again
            predict_dataset = predict_dataset.select(range(args.max_predict_samples))
            print("predict_dataset",predict_dataset)
    # Log a few random samples from the training set:
    #for index in random.sample(range(len(train_dataset)), 3):
        #logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")

    #################
    ## 입력 인코딩을 완료한 데이터셋에 대해서 batch_size 만큼 로드하기 (pytorch 프레임워크의 장점)
    # DataLoaders creation:
    if args.pad_to_max_length:
        # If padding was already done ot max length, we use the default data collator that will just convert everything
        # to tensors.
        data_collator = default_data_collator
    else:
        # Otherwise, `DataCollatorWithPadding` will apply dynamic padding for us (by padding to the maximum length of
        # the samples passed). When using mixed precision, we add `pad_to_multiple_of=8` to pad all tensors to multiple
        # of 8s, which will enable the use of Tensor Cores on NVIDIA hardware with compute capability >= 7.5 (Volta).
        data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=(8 if accelerator.use_fp16 else None))

    ## 실제 평가를 위해서 정답의 근거를 삭제한 형태로 inference
    if args.do_predict:
        predict_dataset_for_model = predict_dataset.remove_columns(["example_id", "offset_mapping"])
        predict_dataloader = DataLoader(
            predict_dataset_for_model, collate_fn=data_collator, batch_size=args.per_device_eval_batch_size
        )


    ## huggingface에서 제공하는 간편하게 평가 결과를 반환하도록 하는 함수 
    ## 깃헙에서 다운로드 받은 utils_qa.py를 참조
    # Post-processing:
    def post_processing_function(examples, features, predictions, stage="eval"):
        # Post-processing: we match the start logits and end logits to answers in the original context.
        predictions = postprocess_qa_predictions(
            examples=examples,
            features=features,
            predictions=predictions,
            version_2_with_negative=args.version_2_with_negative,
            n_best_size=args.n_best_size,
            max_answer_length=args.max_answer_length,
            null_score_diff_threshold=args.null_score_diff_threshold,
            output_dir=args.output_dir,
            prefix=stage,
        )
        # Format the result to the format the metric expects.
        if args.version_2_with_negative:
            formatted_predictions = [
                {"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in predictions.items()
            ]
        else:
            formatted_predictions = [{"id": normalize_answer(k), "prediction_text": normalize_answer(v)} for k, v in predictions.items()]
            print("predictions.items()",predictions.items())
            df_pred = pd.DataFrame(formatted_predictions)
            df_pred.to_csv(args.test_save_path , index=False, encoding='utf-8')

        references = [{"id": ex["id"], "answers": ex[answer_column_name]} for ex in examples]
        print("examples",examples)
        return EvalPrediction(predictions=formatted_predictions, label_ids=references)

    ## F1 score, Exact match
    metric = load_metric("squad_v2" if args.version_2_with_negative else "squad")

    #################
    # Create and fill numpy array of size len_of_validation_data * max_length_of_output_tensor
    def create_and_fill_np_array(start_or_end_logits, dataset, max_len):
        """
        Create and fill numpy array of size len_of_validation_data * max_length_of_output_tensor
        Args:
            start_or_end_logits(:obj:`tensor`):
                This is the output predictions of the model. We can only enter either start or end logits.
            eval_dataset: Evaluation dataset
            max_len(:obj:`int`):
                The maximum length of the output tensor. ( See the model.eval() part for more details )
        """

        step = 0
        # create a numpy array and fill it with -100.
        ## pad에 해당하는 logit 값을 -100으로 지정
        ## 효율적인 배치 계산을 위한 방법으로 지정한 max sequence length에 메모리 효율을 올리는 방법
        logits_concat = np.full((len(dataset), max_len), -100, dtype=np.float64)
        # Now since we have create an array now we will populate it with the outputs gathered using accelerator.gather
        for i, output_logit in enumerate(start_or_end_logits):  # populate columns
            # We have to fill it such that we have to take the whole tensor and replace it on the newly created array
            # And after every iteration we have to change the step

            batch_size = output_logit.shape[0]
            cols = output_logit.shape[1]

            if step + batch_size < len(dataset):
                logits_concat[step : step + batch_size, :cols] = output_logit
            else:
                logits_concat[step:, :cols] = output_logit[: len(dataset) - step]

            step += batch_size

        return logits_concat

    # Prediction
    if args.do_predict:
        all_start_logits = []
        all_end_logits = []
        for step, batch in enumerate(tqdm_notebook(predict_dataloader)):
            with torch.no_grad():
                outputs = model(**batch)
                start_logits = outputs.start_logits
                end_logits = outputs.end_logits

                if not args.pad_to_max_length:  # necessary to pad predictions and labels for being gathered
                    start_logits = accelerator.pad_across_processes(start_logits, dim=1, pad_index=-100)
                    end_logits = accelerator.pad_across_processes(start_logits, dim=1, pad_index=-100)

                all_start_logits.append(accelerator.gather(start_logits).cpu().numpy())
                all_end_logits.append(accelerator.gather(end_logits).cpu().numpy())

        max_len = max([x.shape[1] for x in all_start_logits])  # Get the max_length of the tensor
        # concatenate the numpy array
        start_logits_concat = create_and_fill_np_array(all_start_logits, predict_dataset, max_len)
        end_logits_concat = create_and_fill_np_array(all_end_logits, predict_dataset, max_len)

        # delete the list of numpy arrays
        del all_start_logits
        del all_end_logits

        outputs_numpy = (start_logits_concat, end_logits_concat)
        prediction = post_processing_function(predict_examples, predict_dataset, outputs_numpy)

        print("prediction", prediction)
        print("predictions", prediction.predictions)
        print("predictions_label_ids", prediction.label_ids)
        predict_metric = metric.compute(predictions=prediction.predictions, references=prediction.label_ids)
        logger.info(f"Predict metrics: {predict_metric}")

# Test 를 위한 dataset 로드 및 쿼리 
    if args.do_test:
        print("=================== do_test ===============")
        retrieved_data = []
        with open('final_test_no_context.json', 'r', encoding='utf-8') as f:
            test_dataset = json.load(f)
            for line, retrieved_line in zip(test_dataset['data'], retrieved_data):
 #               prediction2 = post_processing_function(test_dataset, retrieved_data, outputs_numpy)
                prediction2 = post_processing_function(test_dataset, retrieved_data)
                line['context'] = retrieved_line # what you retrieved from document-based database
        with open('final_test_with_retrieved_text.json', 'w', encoding='utf-8') as f:
            json.dump(test_dataset, f)

        print(test_dataset)
        print(retrieved_data)


    if args.output_dir is not None:
        accelerator.wait_for_everyone()
        unwrapped_model = accelerator.unwrap_model(model)
        unwrapped_model.save_pretrained(args.output_dir, save_function=accelerator.save)



In [ ]:
if __name__ == '__main__':
    main()

06/25/2021 11:15:28 - INFO - __main__ -   Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda
Use FP16 precision: False



========== load dataset from HUB ============


06/25/2021 11:15:31 - INFO - datasets_modules.datasets.squad.6b6c4172d0119c74515f44ea0b8262efe4897f2ddb6613e5e915840fdc309c16.squad -   generating examples from = /root/.cache/huggingface/datasets/downloads/b8bb19735e1bb591510a01cc032f4c9f969bc0eeb081ae1b328cd306f3b24008


06/25/2021 11:15:37 - INFO - datasets_modules.datasets.squad.6b6c4172d0119c74515f44ea0b8262efe4897f2ddb6613e5e915840fdc309c16.squad -   generating examples from = /root/.cache/huggingface/datasets/downloads/9d5462987ef5f814fe15a369c1724f6ec39a2018b3b6271a9d7d2598686ca2ff


Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/6b6c4172d0119c74515f44ea0b8262efe4897f2ddb6613e5e915840fdc309c16. Subsequent calls will reuse this data.
DatasetDict({
    train: Dataset({
        features: ['answers', 'context', 'id', 'question', 'title'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['answers', 'context', 'id', 'question', 'title'],
        num_rows: 10570
    })
}) squad plain_text
========== Load Config  ============
Load Config(1)


...

storing https://huggingface.co/bert-base-cased/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/226a307193a9f4344264cdc76a12988448a25345ba172f2c7421f3b6810fddad.3dab63143af66769bbb35e3811f75f7e16b2320e12b7935e216bd6159ce6d9a6
creating metadata file for /root/.cache/huggingface/transformers/226a307193a9f4344264cdc76a12988448a25345ba172f2c7421f3b6810fddad.3dab63143af66769bbb35e3811f75f7e16b2320e12b7935e216bd6159ce6d9a6
06/25/2021 11:15:42 - INFO - filelock -   Lock 140093385481232 released on /root/.cache/huggingface/transformers/226a307193a9f4344264cdc76a12988448a25345ba172f2c7421f3b6810fddad.3dab63143af66769bbb35e3811f75f7e16b2320e12b7935e216bd6159ce6d9a6.lock


loading file https://huggingface.co/bert-base-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/6508e60ab3c1200bffa26c95f4b58ac6b6d95fba4db1f195f632fa3cd7bc64cc.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791
loading file https://huggingface.co/bert-base-cased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/226a307193a9f4344264cdc76a12988448a25345ba172f2c7421f3b6810fddad.3dab63143af66769bbb35e3811f75f7e16b2320e12b7935e216bd6159ce6d9a6
loading file https://huggingface.co/bert-base-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-cased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/ec84e86ee39bfe112543192cf981deebf7e6cbe8c91b8f7f8f63c9be44366158.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
lo

========== Pre processing  0 ============
========== Pre processing  1 ============
column_names ['answers', 'context', 'id', 'question', 'title']



=================== do_test ===============
{'data': [{'answers': {'answer_start': [], 'text': ['None']}, 'context': '', 'id': '1', 'question': 'Who has the Islamic Liberation Party attempted to assassinate? ', 'title': 'Islamism'}, {'answers': {'answer_start': [], 'text': ['None']}, 'context': '', 'id': '2', 'question': 'What distinguishes granal thylakoids?', 'title': 'Chloroplast'}, {'answers': {'answer_start': [], 'text': ['None']}, 'context': '', 'id': '3', 'question': 'Which French kind issued this declaration?', 'title': 'Huguenot'}, {'answers': {'answer_start': [], 'text': ['None']}, 'context': '', 'id': '4', 'question': 'What is the least number of members a board of trustees can have?', 'title': 'United_Methodist_Church'}, {'answers': {'answer_start': [], 'text': ['None']}, 'context': '', 'id': '5', 'question': 'What movement did the fair held in June have its origins in?', 'title': 'Newcastle_upon_Tyne'}, {'answers': {'answer_start': [], 'text': ['None']}, 'context': '', 'i

Configuration saved in result_final/config.json
Model weights saved in result_final/pytorch_model.bin


In [ ]:
# 색인된 엘라스틱서치 데이터베이스에서 'content'를 key로 하여, value에 질의를 입력하여 검색 (여기서는 film을 질의로 사용)
results = es.search(index=index_name, body={'from':0, 'size':10, 'query':{'match':{'content':'film'}}})
for result in results['hits']['hits']:
    print('score:', result['_score'], 'source:', result['_source'])

06/25/2021 11:23:17 - INFO - elasticsearch -   POST http://localhost:9200/test_index/_search [status:200 request:0.011s]


score: 1.8695081 source: {'title': '4', 'content': "The Royal Cinema is an Art Moderne event venue and cinema in Toronto, Canada. It was built in 1939 and owned by Miss Ray Levinsky.\nWhen it was built in 1939, it was called The Pylon, with an accompanying large sign at the front of the theatre. It included a roller-skating rink at the rear of the theatre, and a dance hall on the second floor.\nIn the 1950's the theatre was purchased by Rocco Mastrangelo. In the 1990s, the theatre was renamed 'the Golden Princess'.\nSince early 2007, Theatre D has owned and operated The Royal.\nDuring the daytime it operates as a film and television post-production studio. It hosts film festivals including the European Union Film Festival and Japanese Movie Week.\nThe Royal was featured in the 2013 film The F Word.\nReferences.\n "}


In [ ]:
retrieved_data = []
with open('final_test_no_context.json', 'r', encoding='utf-8') as f:
    test_dataset = json.load(f)
    for line, retrieved_line in zip(test_dataset['data'], retrieved_data):
#               prediction2 = post_processing_function(test_dataset, retrieved_data, outputs_numpy)
        prediction2 = post_processing_function(test_dataset, retrieved_data)
        line['context'] = retrieved_line # what you retrieved from document-based database
with open('final_test_with_retrieved_text.json', 'w', encoding='utf-8') as f:
    json.dump(test_dataset, f)



print(test_dataset['data'][0]['question'])

results = es.search(index=index_name, body={'from':0, 'size':10, 'query':{'match':{'content':test_dataset['data'][50]['question']}}})
#results = es.search(index=index_name, body={'from':0, 'size':2000, 'query':{'match':{'data':test_dataset['data'][50]['question']}}})
for result in results['data']['data']:
    print('score:', result['_score'], 'source:', result['_source'])

06/25/2021 12:52:43 - INFO - elasticsearch -   POST http://localhost:9200/test_index/_search [status:200 request:0.006s]


Who has the Islamic Liberation Party attempted to assassinate? 
score: 1.6558896 source: {'title': '4', 'content': "The Royal Cinema is an Art Moderne event venue and cinema in Toronto, Canada. It was built in 1939 and owned by Miss Ray Levinsky.\nWhen it was built in 1939, it was called The Pylon, with an accompanying large sign at the front of the theatre. It included a roller-skating rink at the rear of the theatre, and a dance hall on the second floor.\nIn the 1950's the theatre was purchased by Rocco Mastrangelo. In the 1990s, the theatre was renamed 'the Golden Princess'.\nSince early 2007, Theatre D has owned and operated The Royal.\nDuring the daytime it operates as a film and television post-production studio. It hosts film festivals including the European Union Film Festival and Japanese Movie Week.\nThe Royal was featured in the 2013 film The F Word.\nReferences.\n "}
{'took': 2, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits':

In [ ]:
print(test_dataset['data'][0]['question'])

Who has the Islamic Liberation Party attempted to assassinate? 


In [ ]:
retrieved_data = []
with open('final_test_no_context.json', 'r', encoding='utf-8') as f:
    test_dataset = json.load(f)
    for line, retrieved_line in zip(test_dataset['data'], retrieved_data):
#               prediction2 = post_processing_function(test_dataset, retrieved_data, outputs_numpy)
        prediction2 = post_processing_function(test_dataset, retrieved_data)
        line['context'] = retrieved_line # what you retrieved from document-based database
with open('final_test_with_retrieved_text.json', 'w', encoding='utf-8') as f:
    json.dump(test_dataset, f)



print(test_dataset['data'][0]['question'])

results = es.search(index=index_name, body={'from':0, 'size':10, 'query':{'match':{'content':test_dataset['data'][50]['question']}}})
#results = es.search(index=index_name, body={'from':0, 'size':50, 'query':{'match':{'data':test_dataset['data'][50]['question']}}})
for result in results['hits']['hits']:
    print('score:', result['_score'], 'source:', result['_source'])

print(results)    

## 👍 실습 종료!! 정말 고생 많으셨습니다.